In [84]:
import pandas as pd
import numpy as np
import nltk 
import matplotlib.pyplot as plt

from gensim.models import Word2Vec

In [85]:
import json

# Define a function to load data from a file
def load_data(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

# Load the data from the three files
test_data_path = 'test_bio_labels.json'
train_data_path = 'train_bio_labels.json'
val_data_path = 'val_bio_labels.json'

test_data = load_data(test_data_path)
train_data = load_data(train_data_path)
val_data = load_data(val_data_path)
# Adjusting approach to properly display the data structure and a sample

# Function to safely extract samples from data which might be a dictionary or list
def get_sample(data):
    if isinstance(data, dict):
        # If the data is a dictionary, return a part of it
        return {k: data[k] for k in list(data)[:2]}
    elif isinstance(data, list):
        # If the data is a list, return the first two elements
        return data[:2]
    else:
        return "Unsupported data type"

# Get samples from each dataset
test_data_sample = get_sample(test_data)
train_data_sample = get_sample(train_data)
val_data_sample = get_sample(val_data)

{
    "test_data_sample": test_data_sample,
    "train_data_sample": train_data_sample,
    "val_data_sample": val_data_sample,
}

{'test_data_sample': {'03f3901e95ed493b866bd7807f623bc0': {'text': "True, our Constitution has no 'due process' clause or the VIII Amendment; but, in this branch of law, after R.C. Cooper v. Union of India, (1970) 1 SCC 248 and Maneka Gandhi v. Union of India, (1978) 1 SCC 248, the consequence is the same.",
   'labels': ['O',
    'O',
    'B_STATUTE',
    'I_STATUTE',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'B_PRECEDENT',
    'I_PRECEDENT',
    'I_PRECEDENT',
    'I_PRECEDENT',
    'I_PRECEDENT',
    'I_PRECEDENT',
    'I_PRECEDENT',
    'I_PRECEDENT',
    'I_PRECEDENT',
    'I_PRECEDENT',
    'I_PRECEDENT',
    'B_PRECEDENT',
    'I_PRECEDENT',
    'I_PRECEDENT',
    'I_PRECEDENT',
    'I_PRECEDENT',
    'I_PRECEDENT',
    'I_PRECEDENT',
    'I_PRECEDENT',
    'I_PRECEDENT',
    'I_PRECEDENT',
    'O',
    'O',
    'O',
    'O',
    'O']},
  'b0311cba3aac4d909eec6e156c059617': {'text': 

In [86]:
import pandas as pd
from nltk.corpus import stopwords as nltk_stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.tokenize import TweetTokenizer
import nltk

# Convert dictionary data to DataFrame
train_df = pd.DataFrame.from_dict(train_data, orient='index').reset_index(drop=True)
test_df = pd.DataFrame.from_dict(test_data, orient='index').reset_index(drop=True)
val_df = pd.DataFrame.from_dict(val_data, orient='index').reset_index(drop=True)

# Display a sample from the train DataFrame to check preprocessing
train_df.head()

,text,labels
0,"Therefore, while interpreting statutory provis...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1,The petitioner in W.P.No.15821 of 2008 was nev...,"[O, O, O, B_CASE_NUMBER, I_CASE_NUMBER, I_CASE..."
2,"The factum of accident, allegation of rash and...","[O, O, O, O, O, O, O, O, O, O, O, O, O, B_OTHE..."
3,..36.. \n\n W.A.No.655/2012 & others Meaning t...,"[O, O, B_CASE_NUMBER, I_CASE_NUMBER, O, O, O, ..."
4,The law on this issue is well settled and the ...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


In [87]:
#convert the dataframe to csv
train_df.to_csv('train_df.csv', index=False)
test_df.to_csv('test_df.csv', index=False)
val_df.to_csv('val_df.csv', index=False)

In [88]:
from gensim.models import Word2Vec
import pandas as pd
import numpy as np

# Prepare sentences for Word2Vec training
train_sentences = train_df['text'].apply(lambda x: x.split()).tolist()
test_sentences = test_df['text'].apply(lambda x: x.split()).tolist()
val_sentences = val_df['text'].apply(lambda x: x.split()).tolist()

# Combine sentences from all datasets
all_sentences = train_sentences + test_sentences + val_sentences

# Train the Word2Vec model
word2vec_model = Word2Vec(sentences=all_sentences, vector_size=512, window=5, min_count=1, sg=0)

# Save the model
word2vec_model.save('word2vec_model')

#example
examples_sentence = train_sentences[0]

# Get the word vectors for the example sentence
example_vectors = [word2vec_model.wv[word] for word in examples_sentence if word in word2vec_model.wv.key_to_index]

# Now similarly, convert all the sentences in train_df to vectors
train_df['vectors'] = train_df['text'].apply(lambda x: [word2vec_model.wv[word] for word in x.split() if word in word2vec_model.wv.key_to_index])
val_df['vectors'] = val_df['text'].apply(lambda x: [word2vec_model.wv[word] for word in x.split() if word in word2vec_model.wv.key_to_index])
test_df['vectors'] = test_df['text'].apply(lambda x: [word2vec_model.wv[word] for word in x.split() if word in word2vec_model.wv.key_to_index])


In [89]:
#print the length of text of first instance in train_df
print(len(train_df['text'][2].split()))
print(len(train_df['vectors'][2]))

18
18


In [90]:
#checking if the length of the vectors is the same as the length of the text for all instances
train_df['len_text'] = train_df['text'].apply(lambda x: len(x.split()))
train_df['len_vectors'] = train_df['vectors'].apply(lambda x: len(x))
train_df['len_text'].equals(train_df['len_vectors'])

#remove the columns
train_df = train_df.drop(columns=['len_text', 'len_vectors'])

In [91]:
# pad the vectors to have the same length
def pad_vectors(vectors, target_length=70):
    # Get the current length of the vectors
    current_length = len(vectors)
    
    # If the current length is less than the target length, pad the vectors
    if current_length < target_length:
        # Calculate the number of vectors to be added
        num_to_add = target_length - current_length
        
        # Add the vectors
        vectors += [np.zeros(512) for _ in range(num_to_add)]
    
    # if the current length is greater than the target length, slice the vectors
    elif current_length > target_length:
        vectors = vectors[:target_length]

    return vectors

# Pad the vectors in the train, test, and val DataFrames
train_df['vectors'] = train_df['vectors'].apply(lambda x: pad_vectors(x, target_length=70))
val_df['vectors'] = val_df['vectors'].apply(lambda x: pad_vectors(x, target_length=70))
test_df['vectors'] = test_df['vectors'].apply(lambda x: pad_vectors(x, target_length=70))

In [92]:
#get all the unique labels
unique_labels = set()
for labels in train_df['labels']:
    unique_labels.update(labels)

In [93]:
print(unique_labels)

{'I_RESPONDENT', 'I_PROVISION', 'B_WITNESS', 'B_CASE_NUMBER', 'B_JUDGE', 'B_PRECEDENT', 'I_PETITIONER', 'I_OTHER_PERSON', 'B_DATE', 'B_GPE', 'I_ORG', 'B_RESPONDENT', 'B_OTHER_PERSON', 'O', 'I_WITNESS', 'B_PROVISION', 'I_PRECEDENT', 'B_STATUTE', 'B_ORG', 'B_PETITIONER', 'I_DATE', 'I_STATUTE', 'I_COURT', 'I_CASE_NUMBER', 'B_COURT', 'I_GPE', 'I_JUDGE'}


In [98]:
#giving all labels index
labels_to_index = {label: index for index, label in enumerate(unique_labels)}
print(labels_to_index)

{'I_RESPONDENT': 0, 'I_PROVISION': 1, 'B_WITNESS': 2, 'B_CASE_NUMBER': 3, 'B_JUDGE': 4, 'B_PRECEDENT': 5, 'I_PETITIONER': 6, 'I_OTHER_PERSON': 7, 'B_DATE': 8, 'B_GPE': 9, 'I_ORG': 10, 'B_RESPONDENT': 11, 'B_OTHER_PERSON': 12, 'O': 13, 'I_WITNESS': 14, 'B_PROVISION': 15, 'I_PRECEDENT': 16, 'B_STATUTE': 17, 'B_ORG': 18, 'B_PETITIONER': 19, 'I_DATE': 20, 'I_STATUTE': 21, 'I_COURT': 22, 'I_CASE_NUMBER': 23, 'B_COURT': 24, 'I_GPE': 25, 'I_JUDGE': 26}


In [ ]:
#pdding the labels as well
def pad_labels(labels, target_length=70):
    # Get the current length of the labels
    current_length = len(labels)
    
    # If the current length is less than the target length, pad the labels
    if current_length < target_length:
        # Calculate the number of labels to be added
        num_to_add = target_length - current_length
        
        # Add the labels
        labels += ['O' for _ in range(num_to_add)]
    
    # if the current length is greater than the target length, slice the labels
    elif current_length > target_length:
        labels = labels[:target_length]

    return labels

In [96]:
# need to write a simple vanilla rnn model to predict the labels
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import MultiLabelBinarizer

# Convert the labels to one-hot encodings
mlb = MultiLabelBinarizer()
mlb.fit(train_df['labels'])

# Transform the labels in the train, test, and val DataFrames
train_labels = mlb.transform(train_df['labels'])
val_labels = mlb.transform(val_df['labels'])
test_labels = mlb.transform(test_df['labels'])

# Define the RNN model
model = models.Sequential([
    layers.Input(shape=(70, 512)),
    layers.SimpleRNN(64, return_sequences=True),
    layers.SimpleRNN(64),
    layers.Dense(64, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(25, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['f1_score'])

# Train the model
history = model.fit(np.array(train_df['vectors'].tolist()), train_labels, validation_data=(np.array(val_df['vectors'].tolist()), val_labels), epochs=10, batch_size=32)

# Evaluate the model
loss, f1_score = model.evaluate(np.array(test_df['vectors'].tolist()), test_labels)

{
    "loss": loss,
    "f1_score": f1_score
}

ImportError: cannot import name 'dtensor' from 'tensorflow.compat.v2.experimental' (c:\Users\amilb\anaconda3\lib\site-packages\tensorflow\_api\v2\compat\v2\experimental\__init__.py)